In [5]:
from dataclasses import dataclass
from vi import Agent, Config, Simulation, Vector2
import pygame as pg
import os

In [6]:
frame_dir = "frames"
os.makedirs(frame_dir, exist_ok=True)
frame_count = 0

def save_frame(screen):
    global frame_count
    pg.image.save(screen, os.path.join(frame_dir, f"frame_{frame_count:05d}.png"))
    frame_count += 1

In [11]:
class RecordingSimulation(Simulation):
    def __init__(self, config):
        super().__init__(config)
        self.frame_count = 0
        os.makedirs("frames", exist_ok=True)

    def after_update(self) -> None:
        # Draw everything to the screen
        self._all.draw(self._screen)

        if self.config.visualise_chunks:
            self.__visualise_chunks()

        # Save current frame as an image
        pg.image.save(self._screen, f"frames/frame_{self.frame_count:05d}.png")

        # Update the screen with the new image
        pg.display.flip()

        self._clock.tick(self.config.fps_limit)

        current_fps = self._clock.get_fps()
        if current_fps > 0:
            self._metrics.fps._push(current_fps)

            if self.config.print_fps:
                print(f"FPS: {current_fps:.1f}")  # noqa: T201

        # Increment a frame counter (you may need to initialize it somewhere)
        self.frame_count += 1


In [12]:
@dataclass
class FlockingConfig(Config):
    alignment_weight: float = 1.1
    cohesion_weight: float = 0.03
    separation_weight: float = 3.1

class FlockingAgent(Agent):
    def change_position(self):
        self.there_is_no_escape()

        neighbors = list(self.in_proximity_accuracy())

        if not neighbors:
            self.pos += self.move 
            return
        
        separationVelocity = Vector2() # move away from nearby agents
        alignmentVelocity = Vector2() # match the average direction of nearby agents
        cohesionVelocity = Vector2() # move toward the average position of nearby agents
        num_neighbors = len(neighbors)

        for neighbor, distance in neighbors: # loop trough each nearby agent.
            offset = self.pos - neighbor.pos # a vector pointing away from the neighbor
            if distance > 0:
                separationVelocity += offset / (distance**1.3) # push neighbors away

            alignmentVelocity += neighbor.move # sum al neighbors movement direction
            cohesionVelocity += neighbor.pos # sum all neigbors position

        # take the average of each vector to get a single direction fro each behavior
        separationVelocity /= num_neighbors
        alignmentVelocity /= num_neighbors
        cohesionVelocity = (cohesionVelocity / num_neighbors - self.pos)

        # multiply bt weights
        separationVelocity *= self.config.separation_weight
        alignmentVelocity *= self.config.alignment_weight
        cohesionVelocity *= self.config.cohesion_weight

        # single movement vector for all behaviors
        self.move = (separationVelocity + alignmentVelocity + cohesionVelocity).normalize() * self.config.movement_speed
        self.pos += self.move

(
    RecordingSimulation(
        FlockingConfig(image_rotation = True, movement_speed = 2.0, radius = 50, seed = 777, duration = 10000, fps_limit = 60)
    )
    .batch_spawn_agents(100, FlockingAgent, images=["images/triangle.png"])
    .run()
)

KeyboardInterrupt: 